<a href="https://colab.research.google.com/github/xinyanz-erin/Applied-Finance-Project/blob/main/Peter's_Code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!curl https://colab.chainer.org/install |sh -
import cupy

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1580  100  1580    0     0  16288      0 --:--:-- --:--:-- --:--:-- 16122
+ apt -y -q install cuda-libraries-dev-10-0
Reading package lists...
Building dependency tree...
Reading state information...
cuda-libraries-dev-10-0 is already the newest version (10.0.130-1).
0 upgraded, 0 newly installed, 0 to remove and 37 not upgraded.
+ pip install -q cupy-cuda100  chainer 
     |████████████████████████████████| 58.9 MB 28 kB/s 
     |████████████████████████████████| 1.0 MB 45.7 MB/s 
+ set +ex
Installation succeeded!


In [5]:
import jax
from jax import random
numsamples = 100000
key = random.PRNGKey(10)
keys = jax.random.split(key, numsamples)
batch_simple = jax.vmap(fast_simple, in_axes = (0,None,None,None,None))
%timeit batch_simple(keys,init_stocks,12,drift,cov).block_unitl_ready()

NameError: ignored

## Portfolio of individual stocks

In [6]:
def optionvalues(key,init_stocks,numsteps,drift,cov,strike):
  return jnp.mean((jnp.maximum(batch_simple(keys,init_stocks,numsteps,drift,cov)[:,-1,:] - strike , 0)))

## Average of Stocks

In [7]:
def optionvalueavg(key,init_stocks,numsteps,drift,cov,strike):
  return (jnp.mean((jnp.maximum(jnp.mean(batch_simple(keys,init_stocks,numsteps,drift,cov)[:,-1,:],axis = 1) - strike , 0))))

In [8]:
gooptionvalue = jax.grad(optionvalue,argmums = 1)

NameError: ignored

In [ ]:
gooptionvalueavg = jax.grad(optionvalueavg,argnums=1)

In [ ]:
drift

In [9]:
key = jax.random.PRNGKey(1)
keys = jax.random.split(key,1000000)
optionvalue(keys,init_stocks,1,drift,cov,110.)

NameError: ignored

In [ ]:
BlackSchoelsCall(100,100,.0,.25,1.)

In [ ]:
init_stocks = jnp.array([100.,100.] * 5)

In [ ]:
Delta(110.,110.,.0,.25,1.)

In [ ]:
Delta(100.,110.,.0,.25,1.)

In [ ]:
BlackScholesCall(*args)

In [ ]:
args = [100.,100.,.01,.25,1.]
v = (BlackScholesCallDelta(*args))
print(v,Delta(*args))

In [ ]:
import numpy as np
def simple_process(key, inital_values,numsteps,drift,cov):
  stocks_init = jnp.zeros((numsteps + 1, inital_values.shape[0]))
  stocks_init = jax.ops.index_update(stocks_init, jax.ops.index[0],initial_values)
  noise = jax.random.multivariate_normal(key, drift,cov,(numsteps+1,))
  sigma = jnp.diag(cov) ** .5
  def time_step(t,val):
    dx = drift + noise[t, :]
    dx2 = jnp.exp((drift - sigma ** 2. / 2.) + noise(t,:))
    val = jax.ops.index_update(val,jax.ops.index[t],(val[t-1] * dx2))
    return val
  return jax.lax.fori_loop(1,numsteps+1,time_step,stocks_init)[1:]